In [ ]:
# Import the modules
import glob

import cv2
from matplotlib import pyplot as plt
import numpy as np

In [ ]:
import matplotlib
matplotlib.rcParams['figure.figsize'] = [12.0, 8.0]
matplotlib.rcParams['image.cmap'] = 'gray'

In [ ]:
# Load the images
filenames = sorted(glob.glob('./data/stigmator/stigmator_*.png'))  # image file names
images = []  # list of images
for filename in filenames:
    images.append(cv2.imread(filename, 0))  # load image in grayscale

In [ ]:
# Find the non-astigmatic image
def get_magnitude_spectrum(img):
    """Calculate the magnitude spectrum of the input image"""
    import numpy as np
    f = np.fft.fft2(img)  # perform FFT
    fshift = np.fft.fftshift(f)  # shift the result so the DC component will be in center
    magnitude_spectrum = np.log(np.abs(fshift) + 1)  # get magnitude spectrum
    return magnitude_spectrum

scores = []
for image in images:
    magnitude_spectrum = get_magnitude_spectrum(image)
    rows, cols = image.shape
    crow, ccol = int(rows / 2), int(cols / 2)
    # divide the magnitude spectrum into the quads
    quads = [magnitude_spectrum[:crow, ccol:], magnitude_spectrum[:crow, :ccol],
             magnitude_spectrum[crow:, :ccol], magnitude_spectrum[crow:, ccol:]]
    # concatenate diagonal quads
    x = np.concatenate((quads[0], quads[2]))
    y = np.concatenate((quads[1], quads[3]))
    # calculate the differences between their sums
    scores.append(np.abs(np.sum(x) - np.sum(y)))

stigmated = np.argmin(scores)  # index of the most stigmated image

In [ ]:
# Show the result
print('The most stigmated is the image {0}'.format(stigmated))
plt.imshow(images[stigmated], cmap='gray')
plt.show()